# Storing/loading data to/from Richmol files

Calculated field-free enegries together with the matrix elements of Cartesian tensor operators (e.g., dipole moment, polarizability) can be stored in Richmol HDF5 files.

We start from generating some field-free rotational energies and matrix elements of dipole moment and polarizability for camphor molecule. For details, see "Rotational dynamics Quickstart":

In [24]:
from richmol.rot import solve
from richmol.rot import LabTensor
from richmol.rot import Molecule

camphor = Molecule()
camphor.XYZ = ("angstrom",
    "O",     -2.547204,    0.187936,   -0.213755,
    "C",     -1.382858,   -0.147379,   -0.229486,
    "C",     -0.230760,    0.488337,    0.565230,
    "C",     -0.768352,   -1.287324,   -1.044279,
    "C",     -0.563049,    1.864528,    1.124041,
    "C",      0.716269,   -1.203805,   -0.624360,
    "C",      0.929548,    0.325749,   -0.438982,
    "C",      0.080929,   -0.594841,    1.638832,
    "C",      0.791379,   -1.728570,    0.829268,
    "C",      2.305990,    0.692768,    0.129924,
    "C",      0.730586,    1.139634,   -1.733020,
    "H",     -1.449798,    1.804649,    1.756791,
    "H",     -0.781306,    2.571791,    0.321167,
    "H",      0.263569,    2.255213,    1.719313,
    "H",      1.413749,   -1.684160,   -1.316904,
    "H",     -0.928638,   -1.106018,   -2.110152,
    "H",     -1.245108,   -2.239900,   -0.799431,
    "H",      1.816886,   -1.883799,    1.170885,
    "H",      0.276292,   -2.687598,    0.915376,
    "H",     -0.817893,   -0.939327,    2.156614,
    "H",      0.738119,   -0.159990,    2.396232,
    "H",      3.085409,    0.421803,   -0.586828,
    "H",      2.371705,    1.769892,    0.297106,
    "H",      2.531884,    0.195217,    1.071909,
    "H",      0.890539,    2.201894,   -1.536852,
    "H",      1.455250,    0.830868,   -2.487875,
    "H",     -0.267696,    1.035608,   -2.160680)

camphor.dip = [1.21615, -0.30746, 0.01140]

camphor.pol = [[115.80434, -0.58739, 0.03276], \
               [-0.58739, 112.28245, 1.36146], \
               [0.03276, 1.36146, 108.47809]]

camphor.frame = "diag(pol)"

sol = solve(camphor, Jmin=0, Jmax=5)

dip = LabTensor(camphor.dip, sol)
pol = LabTensor(camphor.pol, sol)

Here is how we can store and load `Molecule` object

In [25]:
# store molecule camphor
camphor.store('camphor.h5', comment="S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)", replace=True)

# read molecule into camphor2
camphor2 = Molecule()
camphor2.read('camphor.h5')
print("camphor2:", camphor2.__doc__)

# obtain rotational solution for camphor2
sol2 = solve(camphor2, Jmin=0, Jmax=5)

# compare solutions for camphor and camphor2
print("compare solutions")
for J in sol.keys():
    for sym in sol[J].keys():
        for i in range(sol[J][sym].nstates):
            enr = sol[J][sym].enr[i]
            enr2 = sol2[J][sym].enr[i]
            diff = enr - enr2
            print(J, "%4s"%sym, i, "%12.6f"%enr, "%12.6f"%enr2, "%12.6f"%diff)

camphor2: Rigid molecule description, store date: 2021-04-17 11:29:56, comment: S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)
compare solutions
0    A 0     0.000000     0.000000     0.000000
1    A 0     0.076070     0.076070     0.000000
1    A 1     0.084863     0.084863     0.000000
1    A 2     0.087741     0.087741     0.000000
2    A 0     0.227612     0.227612     0.000000
2    A 1     0.234126     0.234126     0.000000
2    A 2     0.242759     0.242759     0.000000
2    A 3     0.269138     0.269138     0.000000
2    A 4     0.269736     0.269736     0.000000
3    A 0     0.453583     0.453583     0.000000
3    A 1     0.457678     0.457678     0.000000
3    A 2     0.474864     0.474864     0.000000
3    A 3     0.497349     0.497349     0.000000
3    A 4     0.500187     0.500187     0.000000
3    A 5     0.548850     0.548850     0.000000
3    A 6     0.548929     0.548929     0.000000
4    A 0     0.752988     0.752988 

To store the field-free solutions, they must first be transformed into a Cartesian tensor form

In [4]:
h0 = LabTensor(camphor, sol)

rchm.store('camphor.h5', h0, comment="S-Camphor field-free energies (in cm^-1)", replace=True)

To inspect which data groups are stored in file, use function `inspect_file`, it returns a dictionary containing all stored data group objects

In [5]:
data_groups = rchm.inspect_file('camphor.h5')

for name, obj in data_groups.items():
    print("data group:", name)
    for elem in vars(obj).keys():
        print("\tattribute:", elem)

print(data_groups['h0'].basis1)

AttributeError: setting energies is not permitted, use PsiTableMK.rotate to assign energies

In [6]:
DIP = LabTensor(camphor.dip, data_groups['h0'].basis1)
print(DIP)

Here is an example of how to aceess the molecular data stored in 'molecule' data group

In [6]:
mol = data_groups['molecule']

print("doc:", mol.__doc__)
print("polarizability:\n", mol.pol)

print("original polarizability:\n", camphor.pol)

doc: 2021-04-16 16:44:19 S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)
polarizability:
 [[ 1.08035306e+02 -8.04509573e-15 -3.48975965e-15]
 [-1.70532612e-14  1.12624870e+02  5.06130530e-15]
 [-5.09201926e-15  8.38035169e-15  1.15904704e+02]]
original polarizability:
 [[ 1.08035306e+02 -8.04509573e-15 -3.48975965e-15]
 [-1.70532612e-14  1.12624870e+02  5.06130530e-15]
 [-5.09201926e-15  8.38035169e-15  1.15904704e+02]]


Here is another example, where we print out some properties of laboratory-frame polarizability tensor

In [7]:
pol2 = data_groups['pol']

print(pol2.__doc__) # date and time when the object was stored
print(pol2.rank) # rank
print(pol2.cart) # list of Cartesian components
print(pol2.os) # list of spherical-tensor indices
print(pol2.Jlist1) # list of J quanta spanned by basis


2021-04-16 16:44:19 S-Camphor polarizability (in atomic units), calculated using DFT


AttributeError: type object 'pol' has no attribute 'rank'

The function `inspect_file` lets us to inspect which data groups are stored in file and which attributes these groups have. To load the complete Cartesian tensor object, including its matrix elements, use `read_tensor`

In [8]:
h0_2 = rchm.read_tensor('camphor.h5', 'h0')
dip_2 = rchm.read_tensor('camphor.h5', 'dip')
pol_2 = rchm.read_tensor('camphor.h5', 'pol')

AttributeError: CarTens attribute 'Jlist1' was not initialized

0 A [[[1.0, 0.0]]]
1 A [[[-0.12601832305446792, 0.028599048581608198], [0.660877953399697, -0.21605664169119018], [0.21762286866220576, 0.6726824079767146]], [[0.0, 0.983159678459981], [0.0, 0.1819882319896128], [0.0, 0.016653229949824183]], [[-0.12601832305446792, -0.028599048581608198], [0.660877953399697, 0.21605664169119018], [0.21762286866220576, -0.6726824079767146]]]
2 A [[[0.04869434664065818, -0.03997126262122468], [0.10904112234592189, -0.06523424409689832], [-0.06599080517164833, -0.11108152841686308], [0.4089335101007364, 0.5622094392111171], [-0.5604975398628166, 0.4070973250563878]], [[0.0543745555422203, 0.22696623231274402], [-0.20721348808629805, -0.6268147043547594], [-0.6618304864644493, 0.21185941746775178], [-0.11882224118192934, 0.050610553635078825], [-0.04435831659823855, -0.09024308774298657]], [[0.9397455994779098, 0.0], [0.30990457819920225, 0.0], [-0.02835849410059156, 0.0], [-0.005249312310428769, 0.0], [0.1414411581036186, 0.0]], [[-0.0543745555422203, 0.2